In [ ]:
#!pip install earthengine-api==0.1.210

In [1]:
import math
import ee
from tqdm import tqdm
import folium
from google.cloud import storage

In [2]:
ee.Initialize()

In [3]:
"""
Copyright (c) 2018 Gennadii Donchyts. All rights reserved.

This work is licensed under the terms of the MIT license.  
For a copy, see <https://opensource.org/licenses/MIT>.

Refactored to Python, Vizzuality, 2020.

"""

def zoomToScale(zoom, tileSize=256):
    equatorial_circumference = 40075016.686 
    tileWidth = equatorial_circumference / math.pow(2, zoom)
    pixelWidth = tileWidth / tileSize
    return pixelWidth


def scaleToZoom(scale, tileSize=256):
    tileWidth = scale * tileSize
    equatorial_circumference = 40075016.686 
    zoom = math.log(equatorial_circumference / tileWidth) / math.log(2)
    return math.ceil(zoom)


def pixelsToMeters(px, py, zoom):
    origin = 2 * math.pi * 6378137 / 2.0
    resolution = zoomToScale(zoom)
    x = px * resolution - origin
    y = py * resolution - origin
    return [x, y]

      
def metersToPixels(x, y, zoom):
    origin = 2 * math.pi * 6378137 / 2.0
    resolution = zoomToScale(zoom)
    px = (x + origin) / resolution
    py = (y + origin) / resolution
    return px, py

def degreesToTiles(lon, lat, zoom):
    tx = math.floor((lon + 180) / 360 * math.pow(2, zoom))
    ty = math.floor((1 - math.log(math.tan(toRadians(lat)) + 1 / math.cos(toRadians(lat))) / math.pi) / 2 * math.pow(2, zoom))
    return [tx, ty]


def tilesToDegrees(tx, ty, zoom):
    lon = tx / math.pow(2, zoom) * 360 - 180
    n = math.pi - 2 * math.pi * ty / math.pow(2, zoom)
    lat = toDegrees(math.atan(0.5 * (math.exp(n) - math.exp(-n))))
    return [lon, lat]


def getTilesForGeometry(geometry, zoom):
    bounds = ee.List(geometry.bounds().coordinates().get(0))
    ll = bounds.get(0).getInfo() # <-- Look at making this happen server-side
    ur = bounds.get(2).getInfo() # <-- Look at making this happen server-side
    tmin = degreesToTiles(ll[0], ll[1], zoom)
    tmax = degreesToTiles(ur[0], ur[1], zoom)
    tiles = []
    for tx in range(tmin[0], tmax[0] + 1):
        for ty in range(tmax[1], tmin[1] + 1):
            bounds = getTileBounds(tx, ty, zoom)
            rect = ee.Geometry.Rectangle(bounds, 'EPSG:3857', False)
            tiles.append(ee.Feature(rect).set({'tx': tx, 'ty': ty, 'zoom': zoom }))   ## <-- we may have problems here with the way the dic is specified
    return ee.FeatureCollection(tiles).filterBounds(geometry)


def getTilesList(geometry, zoom):
    bounds = ee.List(geometry.bounds().coordinates().get(0))
    ll = bounds.get(0).getInfo() # <-- Look at making this happen server-side
    ur = bounds.get(2).getInfo() # <-- Look at making this happen server-side
    tmin = degreesToTiles(ll[0], ll[1], zoom)
    tmax = degreesToTiles(ur[0], ur[1], zoom)
    tiles = []
    for tx in range(tmin[0], tmax[0] + 1):
        for ty in range(tmax[1], tmin[1] + 1):
            bounds = getTileBounds(tx, ty, zoom)
            rect = ee.Geometry.Rectangle(bounds, 'EPSG:3857', False)
            tiles.append(ee.Feature(rect).set({'tx': tx, 'ty': ty, 'zoom': zoom }))
    return tiles

def getTileBounds(tx, ty, zoom, tileSize=256):
    ty = math.pow(2, zoom) - ty - 1 # TMS -> XYZ, flip y index
    tmp_min = pixelsToMeters(tx * tileSize, ty * tileSize, zoom)
    tmp_max = pixelsToMeters((tx + 1) * tileSize, (ty + 1) * tileSize, zoom)
    return [tmp_min, tmp_max]


def toRadians(degrees):
    return degrees * math.pi / 180
 
def toDegrees(radians):
    return radians * 180 / math.pi

In [4]:
def tile_url(image, viz_params=None):
    """Create a target url for tiles for an image.
    e.g.
    im = ee.Image("LE7_TOA_1YEAR/" + year).select("B3","B2","B1")
    viz = {'opacity': 1, 'gain':3.5, 'bias':4, 'gamma':1.5}
    url = tile_url(image=im),viz_params=viz)
    """
    if viz_params:
        d = image.getMapId(viz_params)
    else:
        d = image.getMapId()
    base_url = 'https://earthengine.googleapis.com'
    url = (f"https://earthengine.googleapis.com/v1alpha/{d['mapid']}/tiles/{{z}}/{{x}}/{{y}}")
    return url

## Make some visual previews of areas and tile bounds

In [5]:
brookie_world = [[-166.91564784463128,67.8413000312774],
[-169.453129,64.47279382008166],
[-172.02442436054537,63.783455394136524],
[-170.15625,54.572061655658516],
[-161.015625,50.736455137010665],
[-137.8125,55.3791104480105],
[-130.78125,45.089035564831036],
[-126.5625,28.304380682962783],
[-109.6875,9.102096738726456],
[-86.484375,0],
[-80.859375,-20.632784250388013],
[-81.5625,-53.33087298301705],
[-68.203125,-61.270232790000605],
[-42.890625,-40.44694705960048],
[-21.09375,-4.214943141390639],
[2.8125,-2.108898659243126],
[18.984375,-42.553080288955805],
[37.265625,-37.16031654673676],
[59.0625,-28.30438068296277],
[63.984375,7.013667927566642],
[81.5625,-3.5134210456400323],
[101.953125,-17.308687886770024],
[111.09375,-39.368279149160124],
[156.09375,-50.736455137010644],
[-176.484375,-45.08903556483103],
[-175.078125,-25.16517336866393],
[173.671875,-5.615985819155327],
[145.546875,20.632784250388028],
[168.046875,50.28933925329177],
[179.296875,60.23981116999892],
[-169.62891425000004,65.87472467098549],
[-179.296875,73.02259157147301],
[140.625,79.30263962053658],
[109.6875,83.1948956366159],
[42.890625,82.94032680169508],
[14.0625,81.82379431564338],
[-14.765625,83.1110709962606],
[-49.21875,83.67694304841552],
[-104.0625,81.92318632602199],
[-133.59375,72.3957057065326],
[-168.046875,71.07405646336098],
[-166.91564784463128,67.8413000312774]]

In [105]:
genna_area = [
                [-67.93074032573043,35.46981488813701],
                [-66.1536238287353,36.01338228347171],
                [-64.93898700752635,35.870247281896674],
                [-61.918402961316474,35.690906512945844],
                [-59.34915548456843,32.86102419342828],
                [-58.55125375004707,38.53920148140564],
                [-66.19887770267769,37.7113914931089],
                [-67.16182659937084,35.90904830460234],
                [-70.46072436226257,35.5795448698411],
                [-70.7905135321381,32.96740553173804],
                [-68.40946373914267,30.78468785606502],
                [-67.93074032573043,35.46981488813701]
            ]

In [117]:
area = ee.Geometry.Polygon(brookie_world)

In [111]:
my_tiles3 = getTilesList(area, 3)
my_tiles4 = getTilesForGeometry(area, 4)
my_tiles5 = getTilesForGeometry(area, 3)

In [112]:
url_test_1 = tile_url(my_tiles4, viz_params={'color':"#f5c242"})
#url_test_1 = tile_url(fc, viz_params={'color':"#f5c242"})
url_test_1

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/b309c16667a0eb6d9348fe6ecff791c0-5b8c3cdbdae996f269a1071ce0d60b34/tiles/{z}/{x}/{y}'

In [ ]:
area =  ee.Feature(area)
area_tiles = tile_url(area, {'color':'#03fccf'})
area_tiles

In [113]:
url_test_2 = tile_url(my_tiles5, viz_params={'color':"#ebe460"})
url_test_2

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/f21221b6c4c03a6f91e63c4e1edfbb70-3348a191fdfd32f8e18197f02bdaa610/tiles/{z}/{x}/{y}'

In [96]:
import folium

In [114]:
map_t = folium.Map(tiles='Stamen Toner',zoom_start=6, location=[40, -74])
map_t.add_tile_layer(tiles=url_test_1, attr="Example tiles1")
map_t.add_tile_layer(tiles=url_test_2, attr="Example tiles2")
map_t.add_tile_layer(tiles=area_tiles, attr="Example tiles3")

map_t

## Generate Movie Tiles

Input is an RGB visulized Image Collection

In [15]:
def movie_maker(ic, tile, bucket_name, folder_path):
    props = tile.getInfo().get('properties')
    z = props.get('zoom')
    x = props.get('tx')
    y = props.get('ty')
    g = tile.geometry()
    filtered = ic.filterBounds(g)
    #print(f"Exporting movie-tile to {bucket_name}/{folder_path}/{z}/{x}/{y}.mp4")
    exp_task = ee.batch.Export.video.toCloudStorage(
                    collection = filtered,
                    description = f'Vid {z}_{x}_{y}',
                    bucket= bucket_name,
                    fileNamePrefix = f"{folder_path}/{z}/{x}/{y}",
                    dimensions = [256,256],
                    framesPerSecond = 2,
                    region = g)
    exp_task.start()

In [ ]:
## Original example of EVI from Iker and AJ

# Image Collection
imageCollection = 'MODIS/006/MOD13Q1'
# Start and stop of time series
startDate = ee.Date('2017-01-01')
stopDate  = ee.Date('2018-12-31')
# The zoom levels of the map tiles to export.
# minZoom = 1
# maxZoom = 3
# Image visualization parameters.
visParam = {'bands':['EVI'], 'min':0, 'max':10000, 'palette':[
   'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
   '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
   '012E01', '011D01', '011301'
 ]}

# z_to_px = {0: 156543.00,
#            1: 78271.52,
#            2: 39135.76,
#            3: 19567.88,
#            4: 9783.94,
#            5: 4891.97
#           }

bucket_name = 'skydipper_materials'  # Google Cloud Bucket
folder_path = 'movie-tiles/BTEST'    # Folder path in the bucket

def vis_param(image):
    """Map visualization parameters"""
    return ee.Image(image.visualize(**visParam))

ic = ee.ImageCollection(imageCollection).filterDate(startDate, stopDate)
ic = ic.map(vis_param)

# Uncomment below to generate the tiles at the specified z-levels
# for zlevel in [0, 1, 2, 3]:
#     print("🧨 Calculating Z-level {zlevel}...")
#     tileset = getTilesList(area, zlevel)
#     for tile in tqdm(tileset):
#         movie_maker(ic, tile, bucket_name, folder_path)

In [118]:
# Composite EVI collection Example based on https://developers.google.com/earth-engine/tutorials/community/modis-ndvi-time-series-animation

col = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI') # MODIS NDVI Data
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')  # All world shapes

# Add day-of-year (DOY) property to each image.
def add_DOY_atts(img):
    doy = ee.Date(img.get('system:time_start')).getRelative('day', 'year')
    return img.set('doy', doy)

col = col.map(add_DOY_atts)

# Get a collection of distinct images by 'doy'.
distinctDOY = col.filterDate('2013-01-01', '2014-01-01')

# Define a filter that identifies which images from the complete
# collection match the DOY from the distinct DOY collection.
filter_img = ee.Filter.equals(leftField= 'doy', rightField= 'doy')

# Define a join.
join = ee.Join.saveAll('doy_matches')

# Apply the join and convert the resulting FeatureCollection to an
# ImageCollection.
joinCol = ee.ImageCollection(join.apply(distinctDOY, col, filter_img))

# Apply median reduction among matching DOY collections.
def doyCol(img):
    tmp_i = ee.ImageCollection.fromImages(img.get('doy_matches'))
    return tmp_i.reduce(ee.Reducer.median())

comp = joinCol.map(doyCol)

# Define RGB visualization parameters.
visParams = {
  'min': 0.0,
  'max': 9000.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}


# Create an RGB visualization images for use as animation frames.

def imgViz(img):
    return img.visualize(**visParams)#.clip(mask)

ic = comp.map(imgViz)


bucket_name = 'skydipper_materials'  # Google Cloud Bucket
folder_path = 'movie-tiles/CTEST'    # Folder path in the bucket

for zlevel in [5,6,7,8]:
    print(f"🧨 Calculating Z-level {zlevel}")
    tileset = getTilesList(area, zlevel)
    for tile in tqdm(tileset):
        movie_maker(ic, tile, bucket_name, folder_path)

🧨 Calculating Z-level 5


ServerNotFoundError: Unable to find the server at earthengine.googleapis.com

In [ ]:
# See the progress of the tasks...

#t = ee.batch.Task.list()
#t

## Renaming the `.mp4` files

When exporting the movie-tiles to a Cloud Storage bucket Earth Engine adds a suffix (`ee-export-video` + task_number) at the end of each file name.

To rename the existing files in one of our Cloud Storage buckets:

In [ ]:
from google.cloud import storage
from tqdm import tqdm

In [ ]:
# Rename '.mp4' files
#     for blob in tqdm(blobs):
#         blob_name = blob.name
#         if ("ee-export-video" in blob_name) and (blob_name[-4:] == '.mp4'):
#             new_name = blob_name.split("ee-export-video")[0]+'.mp4'
#             blob = bucket.blob(blob_name)
#             new_blob = bucket.rename_blob(blob, new_name)
#             #print(f'{blob_name} --renamed--> {new_name}')

In [78]:
def reNamer(bucket_name, folder_path, privatekey_path):
    """Renames source files to a clean target that removes jank added by EE."""
    storage_client = storage.Client.from_service_account_json(privatekey_path)
    bucket = storage_client.get_bucket(bucket_name)
    blob_gen = bucket.list_blobs(prefix=folder_path)
    blobs = [blob for blob in blob_gen]
    for blob in tqdm(blobs):
        tmp_name = blob.name
        if tmp_name[-4:] == '.mp4' and ("ee-export-video" in tmp_name):
            target_name = f"{tmp_name.split('ee-export-video')[0]}.mp4"
            #print(f"Renaming {tmp_name} --> {target_name}")
            _ = bucket.rename_blob(blob, target_name)

In [82]:
bucket_name = 'skydipper_materials' 
folder_path = 'movie-tiles/CTEST'  
privatekey_path =  "/Users/Ben/.privateKeys/bucket_changer.json"  # <-- JSON credential file with access to read/write to Skydipper buckets

reNamer(bucket_name, folder_path, privatekey_path)




0it [00:00, ?it/s]


5253it [00:00, 5983.55it/s]


5460it [00:02, 339.30it/s] 


5671it [00:04, 201.77it/s]


5778it [00:05, 179.00it/s]


5886it [00:06, 157.12it/s]


5995it [00:07, 141.95it/s]


6105it [00:08, 139.29it/s]


6216it [00:09, 124.91it/s]


6328it [00:10, 119.84it/s]


6441it [00:11, 104.65it/s]


6555it [00:12, 104.84it/s]


6670it [00:13, 105.85it/s]


6786it [00:14, 112.92it/s]


6903it [00:15, 115.16it/s]


7021it [00:17, 101.55it/s]


7140it [00:18, 105.84it/s]


7260it [00:19, 103.66it/s]


7381it [00:20, 103.97it/s]


7503it [00:21, 113.17it/s]


7626it [00:22, 113.16it/s]


7750it [00:23, 121.90it/s]


7875it [00:24, 128.29it/s]


8001it [00:26, 102.76it/s]


8128it [00:27, 102.15it/s]


8256it [00:28, 113.25it/s]


8385it [00:29, 106.65it/s]


8515it [00:30, 114.65it/s]


8646it [00:31, 121.18it/s]


8778it [00:32, 123.49it/s]


8911it [00:33, 134.13it/s]


9045it [00:33, 144.43it/s]


9180it [00:35, 122.54it/s]


9316it [00:36, 118.65it/s]


9453it [00:37, 12